In [ ]:
import logging
import sys

from astropy.time import Time, TimeDelta
import numpy as np

import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient
from check_utils import check, check_float, check_near

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
# Summit
dataPath="/repo/main"
# NTS
# dataPath="/repo/LATISS"
butler = dafButler.Butler(dataPath, collections=["LATISS/raw/all"], instrument="LATISS")
#client = EfdClient("ncsa_teststand_efd")
client = EfdClient("ldf_stable_efd")
#client = EfdClient("summit_efd")

In [ ]:
obs_date = 20210311
seq_num = 422
dataId = {"instrument": "LATISS", "detector": 0,
          "exposure.day_obs": obs_date, "exposure.seq_num": seq_num}
raw = butler.get('raw', dataId)

In [ ]:
header = raw.getInfo().getMetadata().toDict()
#print(list(header.keys()))
#print(header)

In [ ]:
delta = TimeDelta(10.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai")
obs_end = Time(header["DATE-END"], format="fits", scale="tai")
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
#print(obs_beg)
#print(header["DATE-BEG"])
obs_id = header["OBSID"]
#print(f"IMGTYPE: {header['IMGTYPE']}")
#print(f"EXPTIME: {header['EXPTIME']}")
#print(f"DARKTIME: {header['DARKTIME']}")
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.ATCamera.logevent_startIntegration", "*",
                                               obs_beg, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.ATCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
# Scales are UTC because incoming timestamps are already TAI and astropy screws them up if the scale is TAI.
date_obs = Time(end_image_tel["timestampDateObs"][0], format="unix", scale="utc")
date_end = Time(end_image_tel["timestampDateEnd"][0], format="unix", scale="utc")
date_start_int = Time(start_int["private_sndStamp"][0], format="unix", scale="utc")
date_eoit = Time(end_image_tel["private_sndStamp"][0], format="unix", scale="utc")
print(date_start_int.isot)
print(date_eoit.isot)

In [ ]:
check(header, date_obs.isot, "DATE-BEG")
check(header, date_end.isot, "DATE-END")

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
shutter_motion_profile_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATCamera.logevent_shutterMotionProfile\" WHERE time < \'{date_eoit.isot}Z\' ORDER BY time DESC LIMIT 1")

In [ ]:
check_float(header, start_int["exposureTime"][0], "EXPTIME")
check_float(header, end_image_tel["darkTime"][0], "DARKTIME")
check_float(header, shutter_motion_profile_df["measuredExposureTime"][0], "SHUTTIME")

In [ ]:
current_target_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATPtg.logevent_currentTarget\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
if obs_date >= 20210318:
    mount_positions_start_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATPtg.mountPositions\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
    mount_positions_end_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATPtg.mountPositions\" WHERE time < \'{date_eoit.isot}Z\' ORDER BY time DESC LIMIT 1")
else:
    mount_positions_start_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATPtg.mount_positions\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
    mount_positions_end_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATPtg.mount_positions\" WHERE time < \'{date_eoit.isot}Z\' ORDER BY time DESC LIMIT 1")

In [ ]:
if header["IMGTYPE"] in ["OBJECT", "ENGTEST"]:
    check(header, current_target_df["targetName"][0], "OBJECT")
    check_float(header, np.degrees(current_target_df["ra"])[0], "RA")
    check_float(header, np.degrees(current_target_df["declination"])[0], "DEC")
    check_float(header, current_target_df["rotPA"][0], "ROTPA")
    check_float(header, mount_positions_start_df["ra0"][0], "RASTART")
    check_float(header, mount_positions_end_df["ra0"][0], "RAEND")
    check_near(header, "RA", "RASTART", 0.5)
    check_near(header, "RA", "RAEND", 0.5)
    check_float(header, mount_positions_start_df["declination0"][0], "DECSTART")
    check_float(header, mount_positions_end_df["declination0"][0], "DECEND")
    check_near(header, "DEC", "DECSTART", 0.1)
    check_near(header, "DEC", "DECEND", 0.1)
else:
    print(f"No check on RA/DEC for {header['IMGTYPE']}")

In [ ]:
filter_pos_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATSpectrograph.logevent_reportedFilterPosition\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
grating_pos_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATSpectrograph.logevent_reportedDisperserPosition\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
linear_stage_pos_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATSpectrograph.logevent_reportedLinearStagePosition\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")

In [ ]:
check(header, filter_pos_df["band"][0], "FILTBAND")
check(header, filter_pos_df["name"][0], "FILTER")
check(header, filter_pos_df["slot"][0], "FILTSLOT")
check(header, grating_pos_df["name"][0], "GRATING")
check(header, grating_pos_df["band"][0], "GRATBAND")
check(header, grating_pos_df["slot"][0], "GRATSLOT")
check_float(header, linear_stage_pos_df["position"][0], "LINSPOS")

In [ ]:
focal_plane_info_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATCamera.logevent_focalPlaneSummaryInfo\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
fp_ccd_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATCamera.fp_Ccd\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
vacuum_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.ATCamera.vacuum\" WHERE time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")

In [ ]:
check(header, focal_plane_info_df["ccdManufacturer"][0], "CCD_MANU")
check(header, focal_plane_info_df["ccdType"][0], "CCD_TYPE")
check(header, focal_plane_info_df["ccdManSerNum"][0], "CCD_SERN")
check(header, focal_plane_info_df["ccdLSSTName"][0], "LSST_NUM")
check(header, focal_plane_info_df["ccdSlot"][0], "CCDSLOT")
check(header, focal_plane_info_df["raftBay"][0], "RAFTBAY")
check(header, focal_plane_info_df["sequencerChecksum"][0], "SEQCKSUM")
check(header, focal_plane_info_df["sequencerKey"][0], "SEQNAME")
check(header, focal_plane_info_df["rebLSSTName"][0], "REBNAME")
check(header, focal_plane_info_df["rebSerialNumber"][0], "CONTNUM")
check_float(header, fp_ccd_df["temp"][0], "CCDTEMP")
check_float(header, vacuum_df["tempCCDSetPoint"][0], "TEMP_SET")

In [ ]:
weather_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.WeatherStation.weather\" WHERE WeatherStationID = 1 AND time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
wind_speed_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.WeatherStation.windSpeed\" WHERE WeatherStationID = 1 AND time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
wind_dir_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.WeatherStation.windDirection\" WHERE WeatherStationID = 1 AND time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")
seeing_df = await client.influx_client.query(f"SELECT * from \"efd\".\"autogen\".\"lsst.sal.DIMM.logevent_dimmMeasurement\" WHERE DIMMID = 1 AND time < \'{date_start_int.isot}Z\' ORDER BY time DESC LIMIT 1")

In [ ]:
check_float(header, weather_df["ambient_temp"][0], "AIRTEMP")
check_float(header, weather_df["pressure"][0], "PRESSURE")
check_float(header, weather_df["humidity"][0], "HUMIDITY")
check_float(header, wind_speed_df["value"][0], "WINDSPD")
check_float(header, wind_dir_df["value"][0], "WINDDIR")
check_float(header, seeing_df["fwhm"][0], "SEEING")